In [ ]:
from active_critic.server.analyze import make_acl
import matplotlib.pyplot as plt

In [ ]:
device = 'cpu'
acl, env, expert, seq_len, epsiodes, device = make_acl(device, env_tag='reach', logname='test_plots')
acl.train(epochs=1)

In [ ]:
acl.train_data.reward

In [ ]:
th.all(acl.train_data.obsv[0][1:] == 0)

In [ ]:
acl.policy.eval()
acl.policy.reset()
obs = env.reset()
transitions = sample_expert_transitions(policy=acl.policy.predict, env=env, episodes=1)
attention = acl.policy.critic.attention
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_ylabel('Query')
ax.set_xlabel('Key')
cax = ax.matshow(attention[0].to('cpu'), interpolation='nearest')
fig.colorbar(cax)
plt.show()

In [17]:
import torch as th

def make_step_mask(steps, seq_len, device):
    indices_tensor = th.arange(seq_len, device=device).unsqueeze(0)
    mask = indices_tensor < steps[:, None]
    return mask

def proj_actions(self, org_actions: th.Tensor, new_actions: th.Tensor, steps: int):
    with th.no_grad():
        device = org_actions.device
        step_input = steps + 1
        step_input[step_input == 1] = 0
        mask = make_step_mask(steps=step_input, seq_len=org_actions.shape[1], device=device)

        new_actions[mask] = org_actions[mask]
    return new_actions

batch_size = 3
seq_len = 4
dim = 2

a = th.rand([batch_size, seq_len, dim])
b = th.rand([batch_size, seq_len, dim], requires_grad=True)
steps = th.zeros([batch_size], dtype=bool)
a = proj_actions(self=None, org_actions=b, new_actions=a, steps=steps)
a.retain_grad()

RuntimeError: can't retain_grad on Tensor that has requires_grad=False

In [13]:
loss = ((a-th.ones_like(a))**2).mean()

In [14]:
loss.backward()
a.grad

tensor([[[-0.0796, -0.0142],
         [-0.0500, -0.0380],
         [-0.0789, -0.0299],
         [-0.0714, -0.0559]],

        [[-0.0537, -0.0786],
         [-0.0754, -0.0034],
         [-0.0497, -0.0448],
         [-0.0582, -0.0185]],

        [[-0.0370, -0.0398],
         [-0.0300, -0.0797],
         [-0.0032, -0.0080],
         [-0.0605, -0.0485]]])